Alerta de ordens pendentes criação de KPI:
ordens pendentes, quantidade de itens e informações de clientes para envio de alerta por email e telefone de quem tem o cadastro completo

In [0]:
%python
bronze_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/resource/origem/'
resource_path_volume = '/Volumes/infinitybigdata/azure/bikestore_resource/origem/'

In [0]:
%python
# criar varias tabelas temporárias de forma prática
silver_map = {
    "tmp_silver_customers": f"{silver_path}/customers/",
    "tmp_silver_orders":    f"{silver_path}/orders/",
    "tmp_silver_product":   f"{silver_path}/product/",
}
for view_name, path in silver_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))

In [0]:
describe tmp_silver_orders

In [0]:
WITH pending AS (
  SELECT
  customer_id,
  order_date,
  SUM(quantity) AS quantity,
  store_name,
  status
FROM tmp_silver_orders
WHERE lower(status) = 'pending'
GROUP BY customer_id, store_name, order_date, status
)

SELECT
  p.*,
  c.first_name as first_name_customer,
  c.email,
  c.phone
FROM pending P
LEFT JOIN tmp_silver_customer c ON P.customer_id = c.customer_id
WHERE c.email IS NOT NULL
AND c.phone IS NOT NULL

In [0]:
%python
df_sales_ny_gold = spark.sql(
    """
    WITH pending AS (
SELECT
    customer_id,
    order_date,
    SUM(quantity) AS quantity,
    store_name,
    status
FROM tmp_silver_orders
WHERE lower(status) = 'pending'
GROUP BY customer_id, store_name, order_date, status
)

SELECT
    p.*,
    c.first_name as first_name_customer,
    c.email,
    c.phone
FROM pending P
LEFT JOIN tmp_silver_customer c ON P.customer_id = c.customer_id
WHERE c.email IS NOT NULL
AND c.phone IS NOT NULL
    """
)

# salva em delta
df_orders_pending.write\
    .mode("overwrite")\
    .format("delta")\
    .option("mergeSchema", "true")\
    .save(f"{gold_path}/orders_pending")

In [0]:
CREATE TABLE bikestore.logistics.gold_orders_pending
AS
SELECT *
FROM delta.`abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/gold/orders_pending`;

In [0]:
SELECT *
FROM bikestore.logistics.gold_orders_pending